In [27]:
import torch
import torch.nn as nn
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(5, 2)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
model = SimpleModel()
model.eval()

SimpleModel(
  (fc1): Linear(in_features=10, out_features=5, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=5, out_features=2, bias=True)
)

In [28]:
dummy_input = torch.randn(2, 10)
output = model(dummy_input)
output

tensor([[-0.4834,  0.2183],
        [-0.4611,  0.2798]], grad_fn=<AddmmBackward0>)

In [29]:
# pip install --upgrade onnx onnxscript
onnx_model_path = "simple_model.onnx"
# Export the model to ONNX format
torch.onnx.export(
    model,
    dummy_input,
    onnx_model_path,
)

In [30]:
import onnxruntime
sess = onnxruntime.InferenceSession("simple_model.onnx")
batch = dummy_input.numpy()
input_name = sess.get_inputs()[0].name
pred_onnx, = sess.run(None, {input_name: batch})
pred_onnx

array([[-0.4834043 ,  0.21834402],
       [-0.4610993 ,  0.27978438]], dtype=float32)

In [31]:
torch.onnx.export(
    model,
    dummy_input,
    onnx_model_path,
    dynamo=True,
)
sess = onnxruntime.InferenceSession("simple_model.onnx")
batch = dummy_input.numpy()
input_name = sess.get_inputs()[0].name
pred_onnx, = sess.run(None, {input_name: batch})
pred_onnx

[torch.onnx] Obtain model graph for `SimpleModel([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `SimpleModel([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


array([[-0.4834043 ,  0.21834402],
       [-0.4610993 ,  0.27978438]], dtype=float32)